# LLaMEA Minimal Example

This notebook shows a simple usage of LLaMEA to automatically generate and refine a Python-based optimization algorithm for a toy evaluation.


In [1]:
# Cell 1: Imports
import os
import numpy as np
from llamea import LLaMEA

## Cell 2: Define an evaluation function for LLaMEA

- The function must accept a "solution" argument, which contains code, a name, etc.
- You parse solution.solution (the raw code), dynamically load it, and run it on your problem(s).
- You then set_scores() to record how well it did.

We'll define a simple example with a 1D quadratic function: f(x) = (x - 2)^2
We'll ask the solution code to search for the minimum x in [-5, 5].
We'll then return a score based on how close x is to 2. The closer, the higher the score.

In [2]:
import re
import textwrap
import math

# We implement an exception to stop algorithms that try a too large budget (to prevent infinite loops).
class OverBudgetException(Exception):
    """The algorithm tried to do more evaluations than allowed."""
    pass

def evaluate_simple(solution, explogger=None):
    code_str = solution.solution  # The Python code the LLM generated
    alg_name = solution.name

    # We define our 1D search space: x in [-5, 5], budget=100
    # We'll create a small function that the generated code should optimize.
    def f(x):
        # We only allow so many function calls
        if f.call_count >= 100:
            raise OverBudgetException("Budget exceeded.")
        f.call_count += 1
        return (x - 2.0)**2
    
    f.call_count = 0
    
    # Dynamically run the generated code
    # The code is expected to define a class named alg_name, with __init__(budget, dim) and __call__(f).
    # We'll create a safe execution context to run it.
    safe_globals = {
        "OverBudgetException": OverBudgetException,
        "math": math,
        "np": np,
    }
    local_env = {}
    try:
        exec(code_str, safe_globals, local_env)
    except Exception as e:
        # If there's an error in code, set the score to 0
        solution.set_scores(0, feedback=f"Runtime/Syntax error: {e}")
        return solution
    
    # Instantiate the class with budget=100, dim=1
    try:
        AlgorithmClass = local_env[alg_name]
        algo = AlgorithmClass(budget=100, dim=1)
    except Exception as e:
        solution.set_scores(0, feedback=f"Instantiation error: {e}")
        return solution

    # Now run the algorithm
    best_f = math.inf
    try:
        best_f, best_x = algo(f)
    except OverBudgetException:
        # If over budget, we penalize heavily
        best_f = 9999

    # We'll convert it to a "score" where smaller f is better => we do `score = 1/(1 + best_f)`
    # so that 0 => 1/1 => 1, big f => near 0
    # Note: LLaMEA is optimizing by default! (bigger is better)
    score = 1.0 / (1.0 + best_f)
    
    # Provide feedback
    feedback_str = f"Algorithm {alg_name} got score={score:.4f} (bigger is better)."
    
    # Save the score to the solution object
    solution.set_scores(score, feedback_str)
    return solution


## Cell 3: Create and run the LLaMEA search

We define a small prompt. The LLM will see how we want it to write code (like a class, with __call__).
Then we let LLaMEA iterate a few times, generating and refining solutions.

If you haven't already, set your OpenAI or other API key in your environment variables, e.g.,
`export OPENAI_API_KEY="..."` or `export GEMINI_API_KEY="...."`

You can also use Gemini in most countries for free.

In [3]:
role_prompt = "You are an AI that generates Python optimization code."

task_prompt = textwrap.dedent("""\
Create a Python class named Optimize, with __init__(self, budget, dim), 
and a __call__(self, func) method that tries to find x in [-5, 5] that minimizes func(x).
The function should return the tuple best_f, best_x (value best found and the location).
Implement an efficient algorithm for this task. 
""")

# We'll use a small number of iterations for demonstration
es = LLaMEA(
    f=evaluate_simple,
    n_parents=1, 
    n_offspring=1,
    role_prompt=role_prompt,
    task_prompt=task_prompt,
    experiment_name="my-llamea-example",
    elitism=True,
    model="gpt-4o",  # or e.g. 'gpt-4-turbo'
    api_key="*******",           # Set to your actual key or use environment variable
    budget=3                # Try 3 iterations for a quick demo
)

best_solution = es.run()
print(f"Best found solution: {best_solution.name}, Score={best_solution.fitness:.4f}")
print(f"Generated code:\n{best_solution.solution}")
print(f"Additional feedback: {best_solution.feedback}")


2025-03-03 13:48:48 INFO HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-03 13:48:48 WARNING Message contained no code block
2025-03-03 13:48:48 INFO Started evolutionary loop, best so far: -inf
2025-03-03 13:49:00 INFO HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-03 13:49:00 INFO Generation 2, best so far: 0.0001
2025-03-03 13:49:12 INFO HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-03 13:49:12 INFO Generation 3, best so far: 0.0001


exp-03-03_134845-LLaMEA-gpt-4o-my-llamea-example/log.jsonl


/home/neocortex/.cache/pypoetry/virtualenvs/llamea-84rNYHzd-py3.10/lib/python3.10/site-packages/networkx/algorithms/assortativity/correlation.py:302: RuntimeWarning: invalid value encountered in sqrt
  return float((xy * (M - ab)).sum() / np.sqrt(vara * varb))


Best found solution: Optimize, Score=0.0001
Generated code:
import numpy as np

class Optimize:
    def __init__(self, budget, dim):
        self.budget = budget
        self.dim = dim
        self.bounds = (-5, 5)
        self.num_particles = 30
        self.inertia = 0.5
        self.cognitive = 1.5
        self.social = 1.5

    def __call__(self, func):
        # Initialize particles
        particles = np.random.uniform(self.bounds[0], self.bounds[1], (self.num_particles, self.dim))
        velocities = np.random.uniform(-1, 1, (self.num_particles, self.dim))
        personal_best_positions = particles.copy()
        personal_best_scores = np.full(self.num_particles, float('inf'))
        
        # Global best
        global_best_score = float('inf')
        global_best_position = np.zeros(self.dim)
        
        # Optimization loop
        for _ in range(self.budget):
            # Evaluate current scores
            scores = np.apply_along_axis(func, 1, particles)
          

/home/neocortex/.cache/pypoetry/virtualenvs/llamea-84rNYHzd-py3.10/lib/python3.10/site-packages/networkx/algorithms/assortativity/correlation.py:302: RuntimeWarning: invalid value encountered in scalar divide
  return float((xy * (M - ab)).sum() / np.sqrt(vara * varb))
/home/neocortex/.cache/pypoetry/virtualenvs/llamea-84rNYHzd-py3.10/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:479: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/home/neocortex/.cache/pypoetry/virtualenvs/llamea-84rNYHzd-py3.10/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:480: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
